In [3]:
import os
import zipfile

# Path to the zip file
zip_file_path = r'daily_and_sports_activities.zip'
extract_to_path = r'daily_and_sports_activities'

# Step 1: Extract the zip file
if not os.path.exists(extract_to_path):  # Ensure the directory doesn't already exist
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_path)
        print(f"Extracted zip file to: {extract_to_path}")

# Step 2: List the contents of the extracted directory
print("Contents of extracted directory:", os.listdir(extract_to_path))


Contents of extracted directory: ['data']


In [26]:
#import os

# Update this path with the actual path to your extracted dataset
#base_path = r'C:\Users\dpamu\Downloads\daily_and_sports_activities\data'

# Ensure the base path exists
#if not os.path.exists(base_path):
#    raise FileNotFoundError(f"The specified path does not exist: {base_path}")

# Initialize an empty list to hold all the flattened data
#all_data = []

# Loop through activity folders (a01 to a19)
#for activity in os.listdir(base_path):
#    activity_path = os.path.join(base_path, activity)
#    if os.path.isdir(activity_path): 
#        # Loop through subject folders (p1 to p8)
#        for subject in os.listdir(activity_path):
#            subject_path = os.path.join(activity_path, subject)
#            if os.path.isdir(subject_path): 
#                # Loop through segment files (s01.txt to s60.txt)
#                for segment_file in os.listdir(subject_path):
#                    segment_path = os.path.join(subject_path, segment_file)
#                    #if segment_file.endswith(".txt"): 
#                       # print(f"Processing file: {segment_path}")


In [5]:
import os
import pandas as pd
import numpy as np

# Path to the subject folder containing segment files
subject_folder = r'daily_and_sports_activities/data/a01/p1' # for windows change the / to \ in the path


# List to store all flattened rows
all_flattened_data = []

# Loop through all files in the folder
for file_name in os.listdir(subject_folder):
    if file_name.endswith(".txt"):  # Process only .txt files
        file_path = os.path.join(subject_folder, file_name)
        
        # Load and flatten the file
        data = pd.read_csv(file_path, header=None)
        flattened_data = data.values.flatten()
        all_flattened_data.append(flattened_data)

# Combine all flattened rows into a single DataFrame
combined_df = pd.DataFrame(all_flattened_data)

# Save the combined DataFrame to a CSV file
combined_df.to_csv('flattened_dataset.csv', index=False, header=False)

print(f"All files processed and combined. Shape: {combined_df.shape}")


All files processed and combined. Shape: (60, 5625)


In [6]:
combined_df.head()

,0,1,2,3,4,5,6,7,8,9,...,5615,5616,5617,5618,5619,5620,5621,5622,5623,5624
0,8.0411,1.1530,5.5141,-0.024100,-0.002951,0.007333,-0.78981,-0.072298,0.14805,0.61037,...,-0.039308,-2.7953,-9.0792,2.6194,-0.004393,-0.006382,-0.007844,0.74063,0.30173,-0.055789
1,7.9884,1.1742,5.6439,0.006732,0.015844,-0.003866,-0.79335,-0.074445,0.13338,0.48155,...,-0.039344,-2.7980,-9.0989,2.6300,-0.006768,0.001226,-0.004003,0.74057,0.30237,-0.057019
2,7.8692,1.2414,5.7260,-0.004242,0.003494,0.002519,-0.79502,-0.073527,0.12244,0.47411,...,-0.038984,-2.8134,-9.0845,2.6238,-0.000444,0.004781,-0.009392,0.74024,0.30256,-0.056389
3,7.9589,1.1825,5.6340,-0.010257,0.048934,-0.001143,-0.79292,-0.073751,0.13791,0.54891,...,-0.041509,-2.8111,-9.0954,2.6221,-0.013548,-0.005445,-0.009570,0.74242,0.30413,-0.055288
4,7.8466,1.4019,5.7654,0.004646,0.019555,0.005521,-0.79247,-0.074966,0.12200,0.18988,...,-0.040707,-2.7767,-9.0988,2.6190,0.009631,-0.008417,-0.009028,0.73828,0.30231,-0.057577


### Since the each sec segment sampled at 25Hz we have 125 samples/senssor in 5 sec. This is producing 5625 features. The given instances in 9120 which is not enough to support the given number of features. To mitigate the risk of overfitting, we are reducing the number of features by utilizing only a one-second segment.

In [8]:
import os
import pandas as pd

# Path to the base folder containing all activity folders
base_path = r'daily_and_sports_activities/data' # for windows change the / to \ in the path

#Buckets for the labels
activity_mapping = {
    "a01": "Sitting",
    "a02": "Standing",
    "a03": "Lying on Back",
    "a04": "Lying on Right Side",
    "a05": "Ascending Stairs",
    "a06": "Descending Stairs",
    "a07": "Standing in Elevator",
    "a08": "Moving in Elevator",
    "a09": "Walking in Parking Lot",
    "a10": "Walking on Treadmill (Flat)",
    "a11": "Walking on Treadmill (Inclined)",
    "a12": "Running on Treadmill",
    "a13": "Exercising on Stepper",
    "a14": "Exercising on Cross Trainer",
    "a15": "Cycling (Horizontal)",
    "a16": "Cycling (Vertical)",
    "a17": "Rowing",
    "a18": "Jumping",
    "a19": "Playing Basketball"
}


# List to store all data
all_flattened_data = []
all_activities = []
all_persons = []

#Looping through activities,person, sensors values
for activity in os.listdir(base_path):
    activity_path = os.path.join(base_path, activity)
    if os.path.isdir(activity_path):  
        # Loop through person folders (p1 to p8)
        for person in os.listdir(activity_path):
            person_path = os.path.join(activity_path, person)
            if os.path.isdir(person_path):  
                # Loop through segment files (s01.txt to s60.txt)
                for file_name in os.listdir(person_path):
                    if file_name.endswith(".txt"):  # Process only .txt files
                        file_path = os.path.join(person_path, file_name)
                        
                        # Load/flatten the file
                        data = pd.read_csv(file_path, header=None)
                        data_1sec = data.iloc[:25, :]  # Keeping only 1-second (25 samples)
                        flattened_data = data_1sec.values.flatten()
                        all_flattened_data.append(flattened_data)

                        # Add activity with lables and person info

                        all_activities.append(activity_mapping.get(activity, "Unknown"))
                        all_persons.append(person)



# Combinng all to a single DataFrame
combined_df = pd.DataFrame(all_flattened_data)

# Add activity and person columns
combined_df.insert(0, 'Activity', all_activities)
combined_df.insert(1, 'Person', all_persons)


print(f"Final dataframe with reduced features , activity and person info. Shape: {combined_df.shape}")


# Exclude non-numeric columns for correlation calculation
#numeric_data = combined_df.drop(columns=['Activity', 'Person'])

# Compute the correlation matrix for numeric columns
#correlation_matrix = numeric_data.corr()

#print("Correlation matrix computed and saved.")

#Since all data comprises of sensor measurements it all fields would show correlation
#plt.figure(figsize=(12, 10))
#sns.heatmap(correlation_matrix, cmap='coolwarm', annot=False, fmt=".2f", cbar=True)
#plt.title("Feature Correlation Heatmap")
#plt.show()


Final dataframe with reduced features , activity and person info. Shape: (9120, 1127)


In [9]:
combined_df.head()

,Activity,Person,0,1,2,3,4,5,6,7,...,1115,1116,1117,1118,1119,1120,1121,1122,1123,1124
0,Ascending Stairs,p4,8.4679,-2.7189,0.94689,0.63928,-0.040385,0.067077,-0.36969,0.55460,...,0.55339,-9.1155,-1.8802,-1.15250,-0.015469,0.10051,0.69414,0.54882,0.35652,-0.35006
1,Ascending Stairs,p4,8.1374,-3.2649,0.98650,0.77633,0.089779,0.157790,-0.43743,0.77384,...,0.60235,-9.9356,-1.4720,-0.32292,0.232900,0.19359,0.65006,0.71272,0.32034,-0.42310
2,Ascending Stairs,p4,8.4187,-2.9418,-0.71775,0.56766,-0.322300,0.232190,-0.40913,0.75279,...,0.61076,-10.5810,-2.4220,-0.99518,-0.222600,0.31858,1.00870,0.50753,0.40712,-0.44210
3,Ascending Stairs,p4,7.8393,-2.6960,2.05390,0.34867,0.042874,-0.110610,-0.28866,0.48226,...,0.50679,-9.7747,-1.3506,-2.22290,-0.032205,0.65015,1.13950,0.50347,0.31296,-0.28890
4,Ascending Stairs,p4,9.5991,-2.5011,2.45350,0.11115,-0.240180,0.091841,-0.22087,0.54586,...,0.38264,-6.3365,-4.4395,0.41706,0.208870,0.25913,0.24880,0.18480,0.48054,-0.23865


In [10]:
#Exploring data types in the data frame
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9120 entries, 0 to 9119
Columns: 1127 entries, Activity to 1124
dtypes: float64(1125), object(2)
memory usage: 78.4+ MB


In [11]:
#check for null data
import numpy as np
combined_df.isnull().sum()

Activity    0
Person      0
0           0
1           0
2           0
           ..
1120        0
1121        0
1122        0
1123        0
1124        0
Length: 1127, dtype: int64

In [12]:
combined_df.describe()


,0,1,2,3,4,5,6,7,8,9,...,1115,1116,1117,1118,1119,1120,1121,1122,1123,1124
count,9120.000000,9120.000000,9120.000000,9120.000000,9120.000000,9120.000000,9120.000000,9120.000000,9120.000000,9120.000000,...,9120.000000,9120.000000,9120.000000,9120.000000,9120.000000,9120.000000,9120.000000,9120.000000,9120.000000,9120.000000
mean,7.879943,-0.807308,2.823025,-0.005078,0.022446,-0.002455,-0.601650,0.060835,-0.272478,4.302443,...,0.069614,-7.422868,-3.165476,-0.497145,-0.022566,0.025041,-0.013709,0.461421,0.294218,0.064795
std,5.706403,2.653094,3.667117,0.791649,0.687893,0.334184,0.352953,0.342807,0.374478,5.931536,...,0.364298,5.834603,6.169920,3.389488,0.880451,0.485740,1.167599,0.379462,0.383986,0.359813
min,-13.416000,-26.959000,-18.764000,-11.700000,-10.152000,-4.491300,-1.345900,-1.001400,-1.029900,-19.833000,...,-0.759700,-93.324000,-86.960000,-24.292000,-9.314100,-3.307700,-6.024800,-0.842210,-0.893920,-0.912800
25%,6.981500,-1.503850,0.958283,-0.174487,-0.102540,-0.102270,-0.824767,-0.185420,-0.604215,0.125690,...,-0.215045,-9.816150,-7.914425,-2.179175,-0.315578,-0.125010,-0.401803,0.296325,0.028546,-0.174642
50%,8.869300,-0.400360,2.750500,0.000860,0.018424,-0.002962,-0.690060,0.024050,-0.311025,4.230400,...,0.068505,-8.653450,-2.322350,-0.762290,-0.004837,0.001032,-0.000904,0.511435,0.350645,0.084097
75%,9.713800,0.426432,4.428400,0.160235,0.134960,0.089188,-0.500007,0.285342,0.043585,8.572225,...,0.336905,-2.777000,0.219138,0.415428,0.183945,0.143110,0.590180,0.694597,0.581883,0.299375
max,78.967000,23.190000,67.973000,9.393200,6.852600,7.706400,0.879190,1.029900,0.900980,62.497000,...,1.054500,8.724100,54.195000,19.701000,16.533000,4.995800,4.986600,1.371200,1.120600,1.020600


### Normalizing data by using StandardScaler after splitting the data
### In order  to prevent information leakage data is being split into training and test datasets first and then applying standard scaler
### Dropping the Person column as it is not a relevant feature to train the model on.
### X being the sensors measurements Y is the target variable - Activity

In [13]:

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


X = combined_df.drop(columns=['Activity', 'Person'])
y = combined_df['Activity']

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Applying StandardScaler
scaler = StandardScaler()

# Fitting the scaler on the training set
X_train_scaled = scaler.fit_transform(X_train)

# Transform the test set using the same scaler
X_test_scaled = scaler.transform(X_test)


print(f"Training data shape: {X_train_scaled.shape}")
print(f"Test data shape: {X_test_scaled.shape}")
print(f"Training labels shape: {y_train.shape}")
print(f"Test labels shape: {y_test.shape}")


Training data shape: (7296, 1125)
Test data shape: (1824, 1125)
Training labels shape: (7296,)
Test labels shape: (1824,)


In [14]:
print("X train scaled features\n",X_train_scaled[:5])
print("X test scaled features\n",X_test_scaled[:5])
print("Train activities\n",y_train[:5])
print("Test activities\n", y_test[:5])

X train scaled features
 [[-0.5089416   0.38900197  0.05987206 ...  0.7235761   1.16542482
  -0.85022242]
 [ 1.19221491 -0.31079363 -0.40442956 ...  0.26239454  0.20757117
  -0.95457039]
 [-1.25096928  0.64646811 -2.71988781 ... -0.27293518  0.9572692
   0.50342658]
 [ 0.16216997  0.27214712  0.59627739 ...  0.19748849  0.65930263
  -1.27004227]
 [ 0.1427526   0.23565825 -0.44009642 ...  0.88248767  1.75548491
  -0.99048797]]
X test scaled features
 [[ 0.32654589  0.4455307  -0.55135666 ...  0.06801965 -1.61058104
   0.36034233]
 [ 0.52717455 -0.46386421 -1.2840287  ...  0.58845637  0.39135101
   1.65013785]
 [ 0.08244189  0.53525528 -0.17310374 ... -0.41589751  0.73843295
  -0.79535532]
 [ 0.09158357 -0.80053717  0.10480625 ... -0.14206683 -1.4477467
   1.10707633]
 [ 1.46751231 -2.22894649  0.36646404 ...  0.98156315  0.46523196
   0.8746028 ]]
Train activities
 3725    Walking on Treadmill (Inclined)
170                    Ascending Stairs
8768               Running on Treadmill
426